# IWBDA 2022 SBOL 3 Tutorial

October 2022
This tutorial code goes with the slides at:

https://github.com/SynBioDex/Community-Media/blob/master/2022/IWBDA22/pySBOL3-IWBDA-2022.pptx

Import the module

In [ ]:
from sbol3 import *

Set the default namespace for new objects and create a document

In [ ]:
set_namespace('https://synbiohub.org/public/igem/')
doc = Document()

# Slide 26: GFP expression cassette
Construct a simple part and add it to the Document.

Component
identity: iGEM#I13504
name: "iGEM 2016 interlab reporter"
description: "GFP expression cassette used for 2016 iGEM interlab"
type: SBO:0000251 (DNA)
role: SO:0000804 (Engineered Region)

Which properties are required?  Which properties behave as lists?

In [ ]:
i13504 = Component('i13504', SBO_DNA)
i13504.name = 'iGEM 2016 interlab reporter'
i13504.description = 'GFP expression cassette used for 2016 iGEM interlab study'
i13504.roles.append(SO_NS + '0000804')

Add the GFP expression cassette to the document

In [ ]:
doc.add(i13504)

# Slide 28: expression cassette parts
Here we will create a part-subpart hierarchy.

First, create the RBS component...

Component  
identity: B0034  
name: RBS (Elowitz 1999)  
type: SBO:0000251 (DNA)  
role: SO:0000139 (Ribosome Entry Site)  


In [ ]:
b0034 = Component('B0034', SBO_DNA)
b0034.name = 'RBS (Elowitz 1999)'
b0034.roles = [SO_NS + '0000139']

Next, create the GFP component

identity: E0040  
name: GFP  
type: SBO:0000251 (DNA)  
role: SO:0000316 (CDS)  

In [ ]:
e0040 = Component('E0040', SBO_DNA)
e0040.name = 'GFP'
e0040.roles = [SO_NS + '0000316']

Finally, create the terminator

identity: B0015  
name: double terminator  
type: SBO:0000251 (DNA)  
role: SO:0000141 (Terminator)  

In [ ]:
b0015 = Component('B0015', SBO_DNA)
b0015.name = 'double terminator'
b0015.roles = [SO_NS + '0000141']

Now construct the part-subpart hierarchy

In [ ]:
doc.add(b0034)
doc.add(e0040)
doc.add(b0015)
i13504.features.append(SubComponent(b0034))
i13504.features.append(SubComponent(e0040))
i13504.features.append(SubComponent(b0015))

Slide 30: Location of a SubComponent

Here we add base coordinates to SubCompnents.

But first, assign a sequence to the BBa_I13504 device 

See http://parts.igem.org/Part:BBa_I13504

In [ ]:
i13504_seq = Sequence('i13504_seq')
i13504_seq.elements = 'aaagaggagaaatactagatgcgtaaaggagaagaacttttcactggagttgtcccaattcttgttgaattagatggtgatgttaatgggcacaaattttctgtcagtggagagggtgaaggtgatgcaacatacggaaaacttacccttaaatttatttgcactactggaaaactacctgttccatggccaacacttgtcactactttcggttatggtgttcaatgctttgcgagatacccagatcatatgaaacagcatgactttttcaagagtgccatgcccgaaggttatgtacaggaaagaactatatttttcaaagatgacgggaactacaagacacgtgctgaagtcaagtttgaaggtgatacccttgttaatagaatcgagttaaaaggtattgattttaaagaagatggaaacattcttggacacaaattggaatacaactataactcacacaatgtatacatcatggcagacaaacaaaagaatggaatcaaagttaacttcaaaattagacacaacattgaagatggaagcgttcaactagcagaccattatcaacaaaatactccaattggcgatggccctgtccttttaccagacaaccattacctgtccacacaatctgccctttcgaaagatcccaacgaaaagagagaccacatggtccttcttgagtttgtaacagctgctgggattacacatggcatggatgaactatacaaataataatactagagccaggcatcaaataaaacgaaaggctcagtcgaaagactgggcctttcgttttatctgttgtttgtcggtgaacgctctctactagagtcacactggctcaccttcgggtgggcctttctgcgtttata'
i13504_seq.encoding = IUPAC_DNA_ENCODING
i13504.sequences = [i13504_seq]

Add a Range to the B0015 SubComponent. The base coordinates for B0015 are as follows:

Range  
start: 746  
end: 875  

pySBOL3 does not yet have an easy way to locate features based on arbitrary criteria so we have to loop over the list to find the B0015 SubComponent we are looking for

In [ ]:
for f in i13504.features:
    if f.instance_of == b0015.identity:
        print(f)
        f.locations.append(Range(i13504_seq, 746, 875))
        

# Slide 32: GFP production from expression cassette
In this example, we will create a system representation that includes DNA, proteins, and interactions.

First, create the system representation.  We use the SBO term "functional entity" to represent a system.

Component
identity: i13504_system  
type: SBO:0000236  (functional entity)  

In [ ]:
i13504_system = Component('i13504_system', SBO_NS + '0000236' )
doc.add(i13504_system)

The system has two physical subcomponents, the expression construct and the expressed GFP protein. We already created the expression construct. Now create the GFP protein.

In [ ]:
gfp_protein = Component('GFP', SBO_PROTEIN)
doc.add(gfp_protein)

Now create the part-subpart hierarchy.

In [ ]:
i13504_subcomponent = SubComponent(i13504)
gfp_subcomponent = SubComponent(gfp_protein)
i13504_system.features.append(i13504_subcomponent)
i13504_system.features.append(gfp_subcomponent)

Use a ComponentReference to link SubComponents in a multi-level hierarchy

In [ ]:
e0040_subcomponent = None
for f in i13504.features:
    if f.instance_of == e0040.identity:
        e0040_subcomponent = f
if e0040_subcomponent is None:
    raise Exception()
    
e0040_reference = ComponentReference(i13504_subcomponent, e0040_subcomponent)
i13504_system.features.append(e0040_reference)

Make the Interaction.

Interaction:  
type: SBO:0000589 (genetic production)

Participation:  
role: SBO:0000645 (template)  
participant: e0040_reference  

Participation:  
role: SBO:0000011 (product)  
participant: gfp_subcomponent  

In [ ]:
genetic_production = Interaction([SBO_NS + '0000589'])
template = Participation([SBO_NS + '0000645'], e0040_reference)
product = Participation([SBO_NS + '0000011'], gfp_subcomponent)
i13504_system.interactions.append(genetic_production)


# Finally, write the data out to a file

In [ ]:
doc.write('i13504.nt', file_format=NTRIPLES)